### GT: 실제 데이터에 있는 activations로 바꿔치기

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

from tacotron2.text import _id_to_symbol, _symbol_to_id

data_analysis_dir = "/nas/users/dahye/kw/tts/ttsxai/data_analysis/analyze_prosody"
data = np.load(os.path.join(data_analysis_dir, 'phone_data.npz'), allow_pickle=True)

In [2]:
from ttsxai.utils import jupyter_utils
# jupyter_utils.register_always_reload(packages=[
#     'ttsxai.interfaces',
#     # 'ttsxai.models.text2mel',
#     # 'ttsxai.models.mel2wave',
#     # 'ttsxai.utils',
# ])
jupyter_utils.register_always_reload(modules=[
    # 'ttsxai.text',
    'ttsxai.models.text2mel.tacotron2',
    # 'ttsxai.models.text2mel.fastspeech2',
    # 'ttsxai.utils.plot_utils',
    # 'ttsxai.pitch_calculator',
    # 'ttsxai.energy_calculator'
])

In [3]:
phone_data = {}
for token, phone_data_item in data.items():
    phone_data[token] = phone_data_item.item()

In [6]:
phone_data.keys()

dict_keys(['118', '97', '113', '119', '73', '11', '78', '146', '144', '77', '117', '90', '105', '102', '143', '70', '106', '98', '129', '109', '145', '91', '133', '116', '141', '79', '66', '108', '130', '87', '120', '7', '115', '74', '131', '123', '94', '112', '89', '132', '67', '82', '6', '104', '95', '88', '86', '114', '137', '110', '122', '75', '134', '127', '85', '101', '136', '147', '65', '103', '124', '142', '83', '9', '71', '140', '3', '93', '4', '5', '8', '69', '1', '138', '10', '99', '128', '81', '126', '2'])

In [4]:
import os
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

import torch
import torch.nn as nn
import torch.nn.functional as F

import neurox.interpretation.utils as utils
import neurox.interpretation.ablation as ablation
import neurox.interpretation.linear_probe as linear_probe

from ttsxai.utils.utils import read_ljs_metadata


log_dir = '/nas/users/dahye/kw/tts/ttsxai/logs/probe_tacotron2_duration'
data_activation_dir = "/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow"


In [5]:
import IPython.display as ipd
from matplotlib import pyplot as plt

import torch

from ttsxai.utils.utils import read_ljs_metadata
from ttsxai.utils import plot_utils
from ttsxai.interfaces.tts_interface import TTSInterface, get_text2mel, get_mel2wave

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = '/nas/users/dahye/kw/tts/ttsxai/data/LJSpeech/metadata.csv'
# text_dict = read_ljs_metafile(data_dir)
text_dict = read_ljs_metadata(mode='train')

text2mel = get_text2mel('tacotron2', device)
# mel2wave = get_mel2wave('waveglow', device)
mel2wave = get_mel2wave('hifigan', device)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
text2mel.set_modified_activations({})

In [7]:
text2mel = get_text2mel('tacotron2', device)

In [8]:
tts = TTSInterface(
    device=device,
    text2mel=text2mel,
    mel2wave=mel2wave 
).to(device)

text = text_dict['LJ001-0002']
output_dict = tts.forward(text)
wave = output_dict['wave']

ipd.Audio(wave, rate=tts.sampling_rate)

torch.Size([1, 512, 27])
torch.Size([1, 512, 27])
torch.Size([1, 512, 27])
(80, 170)


In [43]:
output_dict['activations']['lstm'].shape

(27, 512)

In [130]:
alignment = output_dict['alignment']

In [28]:
max_pitch_activation.shape

(2048,)

In [9]:
from ttsxai.articulatory_features import get_articulatory_features_for_phoneme

In [10]:
phonesymbols = [_id_to_symbol[token].replace('@', '') for token in output_dict['token']]
articulatory_features = get_articulatory_features_for_phoneme(phonesymbols)

In [15]:
activations2array(output_dict['activations'])

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         4.1509366e-01,  1.7743040e-03,  1.8924560e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         2.9119179e-01,  5.2845210e-04,  2.0663153e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         2.9987177e-01,  2.7318442e-01, -1.8927841e-01],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -3.6414049e-03,  1.3669996e-04,  2.5776577e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         4.7204041e-01,  1.9804721e-03,  1.2283227e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         4.6959419e-02,  5.6865489e-01, -4.8346270e-02]], dtype=float32)

In [44]:
def activations2array(activations):
    if type(activations) == dict:
        return np.concatenate(list(activations.values()), axis=1)
    elif type(activations) == np.ndarray:
        return activations
    else:
        raise NotImplementedError

In [37]:
def activations2array(activations):
    if type(activations) == dict:
        return np.concatenate(list(activations.values()), axis=1)
    elif type(activations) == np.ndarray:
        return activations
    else:
        raise NotImplementedError

def activations2dict(activations, layer_shapes={'conv_0': 512, 'conv_1': 512, 'conv_2': 512, 'lstm': 512}):
    if type(activations) == dict:
        return activations
    
    if activations.shape[1] != sum(layer_shapes.values()):
        raise ValueError("Total size of layers does not match the array width.")
    
    activations_dict = {}
    current_index = 0
    for layer_name, shape in layer_shapes.items():
        activations_dict[layer_name] = activations[:, current_index:current_index + shape]
        current_index += shape
    
    return activations_dict


new_activations = []

for i, (af, token) in enumerate(zip(articulatory_features, output_dict['token'])):
    # print(_id_to_symbol[token].replace('@', ''))
    # if af == 'Vowel':
    # if False:
    if True:
    # if i > 10:
    # if af != 'Punctuation' and af != 'Space':
    # if af != 'Punctuation':
        token_data = phone_data[str(token)]

        pitch_values = np.array(token_data['duration'])
        activations = np.array(token_data['activations'])

        # # filter
        # valid_ids = pitch_values > 0
        # pitch_values = pitch_values[valid_ids]
        # activations = activations[valid_ids]

        # max_pitch_index = np.argmax(pitch_values)
        # max_pitch_activation = activations[max_pitch_index]
        # new_activations.append(max_pitch_activation)

        # print(_id_to_symbol[token].replace('@', ''), af, pitch_values[max_pitch_index])

        # Calculate the 80th percentile value of the pitch values
        pitch_percentile_80 = np.percentile(pitch_values, 90)

        # Find indices where pitch is in the top 20%
        top_20_percent_indices = np.where(pitch_values >= pitch_percentile_80)[0]

        # # Calculate the 80th percentile value of the pitch values
        # pitch_percentile_80 = np.percentile(pitch_values, 1)

        # # Find indices where pitch is in the top 20%
        # top_20_percent_indices = np.where(pitch_values <= pitch_percentile_80)[0]



        # # # Retrieve the activations for these indices
        top_20_percent_activations = activations[top_20_percent_indices]


        # Define the output activations for comparison
        output_activations = activations2array(output_dict['activations'])[i]

        # Compute the Euclidean distance between output_activations and each of the top 20% activations
        # distances = np.linalg.norm(top_20_percent_activations - output_activations, axis=1)
        distances = np.linalg.norm(top_20_percent_activations[:, -512:] - output_activations[-512:], axis=1)
        
        # Find the index of the activation with the smallest distance
        closest_activation_index = np.argmin(distances)
        # print(distances, distances[closest_activation_index])

        print(token, _id_to_symbol[token].replace('@', ''), af, pitch_values[top_20_percent_indices][closest_activation_index])

        # Retrieve the closest activation
        closest_activation = top_20_percent_activations[closest_activation_index]

        new_activations.append(closest_activation)


        # # Find the index of the lowest pitch within the top 20% activations
        # lowest_pitch_in_top_20_index = np.argmin(pitch_values[top_20_percent_indices])

        # print(_id_to_symbol[token].replace('@', ''), af, pitch_values[top_20_percent_indices][lowest_pitch_in_top_20_index])

        # # Retrieve the activation with the lowest pitch in the top 20%
        # selected_activation = top_20_percent_activations[lowest_pitch_in_top_20_index]

        # new_activations.append(selected_activation)
    else:
        new_activations.append(activations2array(output_dict['activations'])[i])

new_activations = np.array(new_activations)

108 IH0 Vowel 8
119 N Alveolar 9
11   Space 23
88 B Bilabial 9
113 IY1 Vowel 18
108 IH0 Vowel 10
120 NG Velar 10
11   Space 13
116 K Velar 9
73 AH0 Vowel 8
118 M Bilabial 10
129 P Bilabial 8
70 AE1 Vowel 21
130 R Alveolar 12
73 AH0 Vowel 9
133 T Alveolar 10
108 IH0 Vowel 9
143 V Labiodental 8
117 L Alveolar 12
114 IY2 Vowel 18
11   Space 13
118 M Bilabial 13
66 AA1 Vowel 23
90 D Alveolar 8
97 ER0 Vowel 18
119 N Alveolar 9
7 . Punctuation 62


In [38]:
modified_activations = {
    # 'conv_0': torch.from_numpy(activations2dict(new_activations)['conv_0'][None, :]).cuda(),
    # 'conv_1': torch.from_numpy(activations2dict(new_activations)['conv_1'][None, :]).cuda(),
    # 'conv_2': torch.from_numpy(activations2dict(new_activations)['conv_2'][None, :]).cuda(),
    'lstm': torch.from_numpy(activations2dict(new_activations)['lstm'][None, :]).cuda(),
}

text2mel.set_modified_activations(modified_activations)
# text2mel.set_modified_activations({})

# wave = tts.forward(text, alignment=alignment)['wave']
wave = tts.forward(text)['wave']

ipd.Audio(wave, rate=tts.sampling_rate)

torch.Size([1, 512, 27])
torch.Size([1, 512, 27])
torch.Size([1, 512, 27])
### modify lstm
(80, 187)


In [40]:
debug = tts.forward(text)

torch.Size([1, 512, 27])
torch.Size([1, 512, 27])
torch.Size([1, 512, 27])
### modify lstm
(80, 311)


In [43]:
wave = debug['wave']

ipd.Audio(wave, rate=tts.sampling_rate)

In [44]:
1

1

In [180]:
# tts = TTSInterface(
#     device=device,
#     text2mel=text2mel,
#     mel2wave=mel2wave 
# ).to(device)

wave = tts.forward(text, alignment=alignment)['wave']

ipd.Audio(wave, rate=tts.sampling_rate)

torch.Size([1, 512, 27])
torch.Size([1, 512, 27])
torch.Size([1, 512, 27])
(80, 172)


In [489]:
# load pre-trained probe
probe = linear_probe.LinearProbe(512, 1)
probe.load_state_dict(
    torch.load(os.path.join(log_dir, 'models', 'probe_layer_3.pth')))

<All keys matched successfully>

In [491]:
probe(torch.from_numpy(activations2dict(output_dict['activations'])['lstm'][None, :]))

tensor([[[1.0320],
         [1.5649],
         [1.1399],
         [1.5950],
         [1.9607],
         [1.7930],
         [1.6152],
         [1.1400],
         [1.4732],
         [1.6540],
         [1.1687],
         [1.6740],
         [2.2843],
         [2.2464],
         [1.5160],
         [1.3369],
         [1.7331],
         [1.4971],
         [1.3304],
         [1.9861],
         [1.2404],
         [2.1410],
         [2.5059],
         [1.2847],
         [2.5118],
         [2.3610],
         [2.5927]]], grad_fn=<ViewBackward0>)

In [490]:
probe(torch.from_numpy(activations2dict(new_activations)['lstm'][None, :]))

tensor([[[1.1766],
         [1.7833],
         [1.3470],
         [1.7273],
         [2.7022],
         [1.7930],
         [1.8963],
         [1.1790],
         [1.4563],
         [1.6892],
         [1.4970],
         [1.5107],
         [2.6619],
         [2.1394],
         [1.7029],
         [1.3369],
         [1.4841],
         [1.5634],
         [1.9394],
         [2.4792],
         [0.9250],
         [2.1410],
         [2.5357],
         [1.6587],
         [2.5821],
         [2.3610],
         [2.5927]]], grad_fn=<AddBackward0>)